In [1]:
import os
import sqlite3
import re
from datetime import datetime

# Caminho para o banco de dados
DB_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                     'data_warehouse', 'stellar_occultations.db')

import format_data

## 1 - Primeiro passo
Interagir com a Data Base. Buscar objetos e curvas, plotá-las e analisá-las

In [4]:
format_data.get_available_objects()

['1983VD7',
 '1991TF1',
 '1994XM2',
 '1998XA53',
 '1999PY3',
 '1999RU208',
 '1999TA91',
 '1999TB100',
 '1999TG246',
 '1999XR13',
 '1999XW211',
 '2000ET64',
 '2000HT75',
 '2000KJ1',
 '2000OW26',
 '2000SW106',
 '2000TK17',
 '2000WV132',
 '2001HS35',
 '2001US197',
 '2002KX14',
 '2002WC19',
 '2003GG42',
 '2003UP55',
 '2005RM43',
 '2007JJ43',
 '2014EZ51',
 'Abundantia',
 'Academia',
 'Adelheid',
 'Adelinda',
 'Admete',
 'Adorea',
 'Adria',
 'Adzhimushkaj',
 'Aegina',
 'Aegle',
 'Aeternitas',
 'Aethra',
 'Aguntina',
 'Aida',
 'Alauda',
 'Alekto',
 'Alemannia',
 'Aletheia',
 'Alfven',
 'Aline',
 'Alkmene',
 'Alphonsina',
 'Alsatia',
 'Althaea',
 'Amalia',
 'Amalthea',
 'Amata',
 'Ambrosia',
 'Anacostia',
 'Angola',
 'Ani',
 'Antenor',
 'Antigone',
 'Antilochus',
 'Antiope',
 'Antonia',
 'Arctica',
 'Arethusa',
 'Armida',
 'Artemis',
 'Asia',
 'Aspasia',
 'Asplinda',
 'Asterope',
 'Atala',
 'Atalante',
 'Ate',
 'Athamantis',
 'Aulis',
 'Aurelia',
 'Aurora',
 'Ausonia',
 'Austria',
 'Backlunda'

In [ ]:
objects = format_data.get_available_objects()

# Demonstração com o primeiro objeto
selected_object = objects[0]
print(f"\nDatas disponíveis para {selected_object}:")

dates = format_data.get_observation_dates(selected_object)
for date in dates:
    print(f"- {date}")


# Demonstração com a primeira data
selected_date = dates[0]
light_curves = format_data.search_light_curves(selected_object, selected_date)

print(f"\nObservadores para {selected_object} em {selected_date}:")
for lc in light_curves:
    print(f"- {lc.get('observer_name')} (Positivo: {lc.get('is_positive')}, Pontos: {lc.get('point_count')})")

# Demonstração de formato dos dados (primeiras 3 curvas)
x = min(3, len(light_curves))
curves, observers = format_data.format_light_curve_data(x, selected_object, selected_date)


Objetos disponíveis:

Datas disponíveis para 1983VD7:
- 2020-11-01

Observadores para 1983VD7 em 2020-11-01:
- JordiMarco (Positivo: True, Pontos: 1198)


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import cm
import seaborn as sns

def plot_light_curves(curves, observers, object_name, date, save_plots=False):
    """
    Plota as curvas de luz de ocultações estelares.
    
    Args:
        curves (list): Lista de dicionários contendo dados das curvas de luz
        observers (list): Lista com nomes dos observadores correspondentes
        object_name (str): Nome do objeto celeste
        date (str): Data da observação
        save_plots (bool): Se True, salva os plots como arquivos PNG
    """
    
    if not curves:
        print("Nenhuma curva para plotar")
        return
    
    # Configuração do estilo
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    
    # Cores diferentes para cada observador
    colors = cm.tab10(np.linspace(0, 1, len(curves)))
    
    # Figura 1: Todas as curvas normalizadas em um só gráfico
    fig1, ax1 = plt.subplots(figsize=(14, 8))
    
    for i, (curve, observer) in enumerate(zip(curves, observers)):
        time = curve["time"]
        flux_norm = curve["flux_normalized"]
        
        ax1.plot(time, flux_norm, 'o-', color=colors[i], 
                label=f'{observer} ({len(time)} pontos)', 
                linewidth=2, markersize=3, alpha=0.8)
    
    ax1.set_xlabel('Tempo (horas)', fontsize=12)
    ax1.set_ylabel('Fluxo Normalizado', fontsize=12)
    ax1.set_title(f'Curvas de Luz Normalizadas - {object_name} ({date})', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Adiciona linha horizontal em y=1 para referência
    ax1.axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig(f'curvas_normalizadas_{object_name}_{date.replace("-", "")}.png', 
                   dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Figura 2: Curvas originais vs normalizadas (subplots)
    if len(curves) > 1:
        fig2, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # Subplot 1: Curvas originais
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            time = curve["time"]
            flux = curve["flux"]
            
            axes[0].plot(time, flux, 'o-', color=colors[i], 
                        label=f'{observer}', linewidth=2, markersize=3, alpha=0.8)
        
        axes[0].set_xlabel('Tempo (horas)', fontsize=12)
        axes[0].set_ylabel('Fluxo Original', fontsize=12)
        axes[0].set_title(f'Curvas de Luz Originais - {object_name} ({date})', fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()
        
        # Subplot 2: Curvas normalizadas
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            time = curve["time"]
            flux_norm = curve["flux_normalized"]
            
            axes[1].plot(time, flux_norm, 'o-', color=colors[i], 
                        label=f'{observer}', linewidth=2, markersize=3, alpha=0.8)
        
        axes[1].set_xlabel('Tempo (horas)', fontsize=12)
        axes[1].set_ylabel('Fluxo Normalizado', fontsize=12)
        axes[1].set_title(f'Curvas de Luz Normalizadas - {object_name} ({date})', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        axes[1].legend()
        axes[1].axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
        
        plt.tight_layout()
        
        if save_plots:
            plt.savefig(f'comparacao_curvas_{object_name}_{date.replace("-", "")}.png', 
                       dpi=300, bbox_inches='tight')
        
        plt.show()
    
    # Figura 3: Curvas individuais (uma por subplot)
    if len(curves) > 1:
        n_curves = len(curves)
        n_cols = 2 if n_curves > 1 else 1
        n_rows = (n_curves + n_cols - 1) // n_cols
        
        fig3, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5*n_rows))
        if n_curves == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            row = i // n_cols
            col = i % n_cols
            ax = axes[row, col] if n_rows > 1 else axes[col]
            
            time = curve["time"]
            flux_norm = curve["flux_normalized"]
            
            ax.plot(time, flux_norm, 'o-', color=colors[i], 
                   linewidth=2, markersize=4, alpha=0.8)
            
            ax.set_xlabel('Tempo (horas)', fontsize=10)
            ax.set_ylabel('Fluxo Normalizado', fontsize=10)
            ax.set_title(f'{observer} - {len(time)} pontos', fontsize=12, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
        
        # Remove subplots vazios
        for i in range(len(curves), n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            ax = axes[row, col] if n_rows > 1 else axes[col]
            ax.remove()
        
        plt.suptitle(f'Curvas Individuais - {object_name} ({date})', fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        if save_plots:
            plt.savefig(f'curvas_individuais_{object_name}_{date.replace("-", "")}.png', 
                       dpi=300, bbox_inches='tight')
        
        plt.show()


def plot_statistics(curves, observers, object_name, date):
    """
    Plota estatísticas das curvas de luz.
    
    Args:
        curves (list): Lista de dicionários contendo dados das curvas de luz
        observers (list): Lista com nomes dos observadores correspondentes
        object_name (str): Nome do objeto celeste
        date (str): Data da observação
    """
    
    if not curves:
        print("Nenhuma curva para analisar")
        return
    
    # Calcula estatísticas
    stats = []
    for curve, observer in zip(curves, observers):
        flux_norm = curve["flux_normalized"]
        stats.append({
            'observer': observer,
            'n_points': len(flux_norm),
            'min_flux': min(flux_norm),
            'max_flux': max(flux_norm),
            'mean_flux': np.mean(flux_norm),
            'std_flux': np.std(flux_norm),
            'range_flux': max(flux_norm) - min(flux_norm)
        })
    
    # Gráfico de barras com estatísticas
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    observers_short = [obs[:15] + '...' if len(obs) > 15 else obs for obs in observers]
    
    # Número de pontos
    axes[0, 0].bar(observers_short, [s['n_points'] for s in stats], color='skyblue')
    axes[0, 0].set_title('Número de Pontos por Observador', fontweight='bold')
    axes[0, 0].set_ylabel('Número de Pontos')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Amplitude (range) do fluxo
    axes[0, 1].bar(observers_short, [s['range_flux'] for s in stats], color='lightcoral')
    axes[0, 1].set_title('Amplitude do Fluxo Normalizado', fontweight='bold')
    axes[0, 1].set_ylabel('Amplitude (Max - Min)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Desvio padrão
    axes[1, 0].bar(observers_short, [s['std_flux'] for s in stats], color='lightgreen')
    axes[1, 0].set_title('Desvio Padrão do Fluxo', fontweight='bold')
    axes[1, 0].set_ylabel('Desvio Padrão')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Fluxo médio
    axes[1, 1].bar(observers_short, [s['mean_flux'] for s in stats], color='gold')
    axes[1, 1].set_title('Fluxo Médio Normalizado', fontweight='bold')
    axes[1, 1].set_ylabel('Fluxo Médio')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].axhline(y=1, color='red', linestyle='--', alpha=0.7, linewidth=1)
    
    plt.suptitle(f'Estatísticas das Curvas - {object_name} ({date})', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Imprime tabela de estatísticas
    print(f"\n{'='*80}")
    print(f"ESTATÍSTICAS DAS CURVAS - {object_name} ({date})")
    print(f"{'='*80}")
    print(f"{'Observador':<20} {'N Pontos':<10} {'Min':<8} {'Max':<8} {'Média':<8} {'Desvio':<8} {'Amplitude':<10}")
    print(f"{'-'*80}")
    
    for s in stats:
        print(f"{s['observer']:<20} {s['n_points']:<10} {s['min_flux']:<8.4f} {s['max_flux']:<8.4f} "
              f"{s['mean_flux']:<8.4f} {s['std_flux']:<8.4f} {s['range_flux']:<10.4f}")


# Exemplo de uso:
# Assumindo que você já tem as variáveis curves, observers, object_name e date
# plot_light_curves(curves, observers, object_name, date, save_plots=True)
# plot_statistics(curves, observers, object_name, date)

ImportError: cannot import name 'cm' from 'matplotlib.colors' (c:\Users\thiago.cunha\.conda\envs\Astro\Lib\site-packages\matplotlib\colors.py)

## 2 - Segundo passo
Ler uma amostragem grande de curvas (+100), programar um método de recortar trechos negativos e associá-los a curvas separadas negativas. 

Ideia: separar recortes em outra variável "fake_negative".

## 3 - Terceiro passo
Organizar variáveis de curvas lidas, aplicar feature engineering e criar DataSet de treino/validação.